# Job Classifier

In [1451]:
import pandas as pd
import numpy as np
from nltk.tag import StanfordPOSTagger

In [1452]:
import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

## Prepare sample data

In [1453]:
inputfile = "./tags/nnp-summary.txt"

#df_nn_summary = pd.read_table(nn_summary, sep="|",index_col=False, header=None, error_bad_lines=False)

#names = ["token"]*1314
#df_nn_summary = pd.read_table(nn_summary, sep="|",index_col=False, header=None, names = names) #error_bad_lines=False)

In [1454]:
import csv
with open(inputfile) as f:
    reader = csv.reader(f, delimiter="|")
    d = list(reader)
print d[0][2] # test

ETL


In [1455]:
# the first line of our data
print '|'.join(d[0])

SQL|SQL Developer|ETL|SSIS|SQL|Server|SQL|Server|Web|JavaScript|JQuery|HTML5|CSS3|JavaScript|•|Extensive|Microsoft|Business|Intelligence|SQL|Server|Integration|Services|SSIS|SQL|Server|Reporting|Services|SSRS|ETL|SSIS|Custom|Report|Tabular|Reports|Matrix|Reports|Ad|SQL|Server|Reporting|Services|SSRS|DATA|•|Expert|Bulk|BCP|Data|Transformation|Services|DTS|SSIS|•|Expertise|SQL|Server|Data|Modeling|Star|Schema/Snowflake|Schema|Fact|Dimensions|Visio|Software|Development|Life|Cycle|SDLC|Agile|Methodology|Waterfall|ETL|SSIS|•|Extensive|SSIS|File|Transfer|Protocol|FTP|Secure|File|Transfer|Protocol|SFTP


In [1456]:
# TECH category list
d_target = [item[0] for item in d]
print d_target[0:10]
print len(d_target)

['SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL']
1271


In [1457]:
# TITLE category list
d_title = [' '.join(item[1:2]) for item in d]
print d_title[0:10]
print len(d_title)

['SQL Developer', 'SQL Developer', 'SQL / BI Developer', 'Sr. SQL Developer', 'Sr. SQL Server DBA', 'Sr. SQL Developer', 'PL/SQL Developer', 'Sr. SQL Server DBA', 'SQL Server DBA', 'Sql Server/bi Developer']
1271


In [1458]:
# token list
d_train = [item[2:] for item in d]
print d_train[0][:10]
print len(d_train)

['ETL', 'SSIS', 'SQL', 'Server', 'SQL', 'Server', 'Web', 'JavaScript', 'JQuery', 'HTML5']
1271


## Read training data from MongoDB
files are in JSON format

In [1]:
import pymongo
client = pymongo.MongoClient('192.168.1.232',27017)

In [3]:
db = client.ir

In [4]:
db.collection_names()

[u'movie',
 u'TaggedResumes',
 u'OperatingSystems',
 u'Jobs',
 u'Mails',
 u'KeyWords',
 u'IndeedJobs',
 u'TaggedMails',
 u'ResumeLinks',
 u'Technologies']

In [5]:
resumes = db.TaggedResumes

In [11]:
resumes.find_one()

{u'_id': ObjectId('57f5dc3452331e201cb425c2'),
 u'details': {u'additionalInfo': u'Technical Skills \r\n \r\nLanguages: Transact-SQL, Dynamic SQL, C, C++, Visual Basic 6.0, HTML, CSS, JavaScript. \r\nRDBMS: SQL Server [\u2026] MS Access. \r\nDatabase Design Tools and \r\nData Modeling: MS Visio, UML, Star Schema, FACT & Dimensions tables. \r\n \r\nDWH / BI Tools: SQL Server Reporting Services (SSRS), SQL Server Analysis Services (SSAS), SQL Server Integration Services (SSIS), Business Intelligence Development Studio (BIDS)/Data Tools, Visual Studio. \r\nTools and Utilities: SQL Server Management Studio, SQL Server Enterprise Manager, SQL Server Profiler, SQL Query Analyzer, Import & Export Wizard, SQL Server 2012 Query Editor, Solution Explorer, Analysis Manager, Reporting Server, Business Intelligence Development Studio (BIDS)/Data Tools, Web Services. \r\n \r\nOperating Systems: Windows [\u2026] NT & Windows 9x and MS-DOS',
  u'detailTags': [{u'tagLabel': u'NNP', u'wordToken': u'Techn

In [18]:
aa =resumes.find({},{"details.title": 1, "details.summary": 1})

In [19]:
aa[0]

{u'_id': ObjectId('57f5dc3452331e201cb425c2'),
 u'details': {u'summary': u'\u2022 Over 6 years of Experience in requirements gathering, design, development, implementation and testing the web applications and ETL tools which includes SSIS, SQL Server 2012, and SQL Server 2008. \r\n\u2022 Experience in documenting the business requirements for the development team. \r\n\u2022 Experience in Web design, development, testing/debugging, implementation, and operations in medium-to-large enterprise environments \r\n\u2022 Experienced in JavaScript, JQuery, HTML5, CSS3, and JavaScript. \r\n\u2022 Extensive experience in Microsoft Business Intelligence technologies like SQL Server Integration Services (SSIS) and SQL Server Reporting Services (SSRS). \r\n\u2022 Worked on all activities related to the development, implementation, administration and support of ETL processes for large-scale Data Warehouses using SSIS 2012. \r\n\u2022 Experience in developing Custom Report and different types of Tab

## Processing input data : job requirement.docx/doc

Python_job_req1.txt

In [1465]:
### STEP1: read in txt
#with open('./Python_job_req1.txt', 'r') as myfile: # python
#with open('./java-developer-req1.txt', 'r') as myfile: # java
with open('./sql-req1.txt', 'r') as myfile: # SQL
    text=myfile.read().replace('\n', ' ')

In [1466]:
#text = text.replace("\x95\t"," ")

In [1467]:
tokens = nltk.word_tokenize(text.decode('UTF-8'))
print ' '.join(tokens[:20])

SQL Developer Indus Software Solutions Inc. - Pennington , NJ $ 70 an hour - Contract Job Title : SQL


In [1468]:
input_postag = nltk.pos_tag(tokens)
input_postag[:11]

[(u'SQL', 'NNP'),
 (u'Developer', 'NNP'),
 (u'Indus', 'NNP'),
 (u'Software', 'NNP'),
 (u'Solutions', 'NNP'),
 (u'Inc.', 'NNP'),
 (u'-', ':'),
 (u'Pennington', 'NNP'),
 (u',', ','),
 (u'NJ', 'NNP'),
 (u'$', '$')]

In [1469]:
# filtering list, keep NNP only
input_postag_nn = [x[0] for x in input_postag if x[1] == 'NNP']
#input_postag_nn = list(set(input_postag_nn))
print '|'.join(input_postag_nn)

SQL|Developer|Indus|Software|Solutions|Inc.|Pennington|NJ|Job|Title|SQL|Developer|Location|Pennington|NJ|Duration|Pay|W2|MUST|WORK|ON|W2|NO|CORP|TO|CORP|Description|Bank|America|Sr.|SQL|Developer|Familiar|Skill|DB2|Informix|Tandem|SQL|DDBMS|Oracle|Teradata|RDBMS|Sybase|SQL|Server|Assembler|CICS|IMS|Network|Telecommunication|Applications|Security|Internet|Database|Wireless|Enterprise|Data|Warehousing|Teradata|RDBMS|Candidate|SQL|Server|R2|Cluster|Always|ON|Hrinish|Patalay|Indus|Software|Solutions|Inc|C|D|Job|Type|Salary|Bachelor|SQL|Developer|IT


In [1470]:
# generate bigram for test data
#def ngrams(input, n):
#  output = []
#  for i in range(len(input)-n+1):
#    output.append(input[i:i+n])
#  return output

#input_postag_2grams = [' '.join(x) for x in ngrams(input_postag_nn,2)]
#input_postag_nn = input_postag_nn + input_postag_2grams

## Classification using Naive Bayes in scikit-learn 
[Working With Text](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#working-with-text-data)  
[Multinomial Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)  
[MultinomialNB Example](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)

In [1471]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [1472]:
d_train_l = [" ".join(item) for item in d_train]

In [1473]:
#d_train_l

In [1474]:
input_nn = " ".join(input_postag_nn)

In [1475]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(ngram_range=(1, 2))#(stop_words = 'english')

In [1476]:
#X_vectorized = vectorizer.fit_transform(X_nn_summary_list)
d_train_l_vector = vectorizer.fit_transform(d_train_l)

In [1477]:
print d_train_l_vector.shape
print len(d_target)

(1271, 41473)
1271


In [1478]:
#print vectorizer.get_feature_names()

In [1479]:
vectorizer.vocabulary_.get(u'python')

28170

In [1480]:
#from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer = TfidfTransformer(use_idf=False).fit(d_train_l_vector)
#d_train_tf = tf_transformer.transform(d_train_l_vector)
#d_train_tf.shape

In [1481]:
tfidf_transformer = TfidfTransformer()
d_train_tfidf = tfidf_transformer.fit_transform(d_train_l_vector)
print d_train_tfidf.shape
print len(d_target),type(d_target)

(1271, 41473)
1271 <type 'list'>


In [1482]:
d_train_l_vector.toarray()[0].sum()

163

In [1483]:
# input_nn = " ".join(tokens)  # comment out this line if you want to use nnp instead of all test corpus
# vectorize test data with nnp only
input_nn_list = [input_nn]
input_nn_vectorized = vectorizer.transform(input_nn_list)

In [1484]:
# test with data from training set
testString = d_train_l[0].replace("\xe2\x80\xa2","")
testStringList = [testString]
testVector = vectorizer.transform(testStringList)
testVector2 = vectorizer.transform(["This is a job requirement for python programmer\
    who needs to know flask, numpy and pandas."])

In [1485]:
# fit multinomial Naive Bayes model
clf1 = MultinomialNB()
clf2 = MultinomialNB()
clf3 = MultinomialNB()

model1 = clf1.fit(d_train_l_vector, d_target) # TECH model
# validate model using training data
print model1.predict(testVector),model1.predict(testVector2) # Python # SQL

model2 = clf2.fit(d_train_tfidf, d_target) # TECH model w/ df-idf
print model2.predict(testVector),model2.predict(testVector2) # Python # SQL

model3 = clf3.fit(d_train_tfidf, d_title) # TITILE model w/ df-idf
print model3.predict(testVector),model3.predict(testVector2) # Python # SQL

['SQL'] ['Python']
['SQL'] ['Python']
['Net Developer'] ['Python Developer']


In [1486]:
# predict test data
print "Model1-Tech1:", model1.predict(input_nn_vectorized)
print "Model2-Tech2:", model2.predict(input_nn_vectorized)
print "Model3-Title:", model3.predict(input_nn_vectorized)

Model1-Tech1: ['ETL']
Model2-Tech2: ['PL/SQL']
Model3-Title: ['Oracle PL/SQL Developer']


In [1487]:
# test data, 2grams is not shown here. 2grams are generated when calling vectorizer
print " ".join(tokens[:2]), "|","|".join(input_nn.split(" "))

SQL Developer | SQL|Developer|Indus|Software|Solutions|Inc.|Pennington|NJ|Job|Title|SQL|Developer|Location|Pennington|NJ|Duration|Pay|W2|MUST|WORK|ON|W2|NO|CORP|TO|CORP|Description|Bank|America|Sr.|SQL|Developer|Familiar|Skill|DB2|Informix|Tandem|SQL|DDBMS|Oracle|Teradata|RDBMS|Sybase|SQL|Server|Assembler|CICS|IMS|Network|Telecommunication|Applications|Security|Internet|Database|Wireless|Enterprise|Data|Warehousing|Teradata|RDBMS|Candidate|SQL|Server|R2|Cluster|Always|ON|Hrinish|Patalay|Indus|Software|Solutions|Inc|C|D|Job|Type|Salary|Bachelor|SQL|Developer|IT


## Classifiying using Random Forest

In [1488]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier
forest1 = RandomForestClassifier(n_estimators = 100) #random_state=15326
forest2 = RandomForestClassifier(n_estimators = 100)
forest3 = RandomForestClassifier(n_estimators = 100)

Training the random forest...


In [1489]:
forest1 = forest1.fit(d_train_l_vector, d_target)
forest2 = forest2.fit(d_train_tfidf, d_target)
forest3 = forest3.fit(d_train_tfidf, d_title)

In [1491]:
# validate model with data from training set
print forest1.predict(testVector), forest1.predict(testVector2)  #SQL Python

['SQL'] ['Python']


In [1490]:
# test data
print "Forest1-Tech1:",forest1.predict(input_nn_vectorized)
print "Forest2-Tech2:",forest2.predict(input_nn_vectorized)
print "Forest3-Title:",forest3.predict(input_nn_vectorized)

Forest1-Tech1: ['ETL']
Forest2-Tech2: ['ETL']
Forest3-Title: ['Premier SQL Support Engineer At Microsoft']
